# EASI-FISH pipeline tutorial
---

This tutorial shows the basic requirements for running the pre-packaged easi-fish registration pipeline. Although we will only be running a few functions in this tutorial, a lot of complex things are happening behind the scenes. At the end of the tutorial I will recommend another tutorial and more reading to learn about those behing-the-scenes components.

## Define inputs
---

We will need a low resolution and high resolution datasets for our fixed and moving images. The low resolution datasets will be used for global alignment and the high resolution datasets will be used for local alignment corrections. It's assumed that the datasets do not fit into memory so they are all "lazy loaded" with zarr. Of course, we also need to know the voxel sampling rate in microns, so that is extracted from the image file metadata.

In [ ]:
# load all data
import numpy as np
import zarr, tifffile

# file paths to tutorial data N5 files
# replace capitalized text in paths below with the location where you cloned bigstream
fix_path = '/PATH/TO/BIGSTREAM/REPOSITORY/resources/fix.n5'
mov_path = '/PATH/TO/BIGSTREAM/REPOSITORY/resources/mov.n5'
fix_zarr = zarr.open(store=zarr.N5Store(fix_path), mode='r')
mov_zarr = zarr.open(store=zarr.N5Store(mov_path), mode='r')

# get pointers to the low res scale level
fix_lowres = fix_zarr['/lowres']
mov_lowres = mov_zarr['/lowres']

# we need the voxel spacings for the low res data sets
fix_meta = fix_lowres.attrs.asdict()
mov_meta = mov_lowres.attrs.asdict()
fix_lowres_spacing = np.array(fix_meta['pixelResolution']) * fix_meta['downsamplingFactors']
mov_lowres_spacing = np.array(mov_meta['pixelResolution']) * mov_meta['downsamplingFactors']
fix_lowres_spacing = fix_lowres_spacing[::-1]  # put in zyx order
mov_lowres_spacing = mov_lowres_spacing[::-1]

# sanity check: print the voxel spacings and lowres dataset shapes
print(fix_lowres_spacing, mov_lowres_spacing)
print(fix_lowres.shape, mov_lowres.shape)

# get pointers to the high res scale level
fix_highres = fix_zarr['/highres']
mov_highres = mov_zarr['/highres']

# we need the voxel spacings for the high res data sets
fix_meta = fix_highres.attrs.asdict()
mov_meta = mov_highres.attrs.asdict()
fix_highres_spacing = np.array(fix_meta['pixelResolution']) * fix_meta['downsamplingFactors']
mov_highres_spacing = np.array(mov_meta['pixelResolution']) * mov_meta['downsamplingFactors']
fix_highres_spacing = fix_highres_spacing[::-1]
mov_highres_spacing = mov_highres_spacing[::-1]

# sanity check: print the voxel spacings and lowres dataset shapes
print(fix_highres_spacing, mov_highres_spacing)
print(fix_highres.shape, mov_highres.shape)

# write data to view in fiji or similar
# here we write the highres data to disk, which requires loading it all into memory
# this is fine for tutorial data, but in practice you would not do this with your
# large data
tifffile.imsave('./fix_lowres_data.tiff', fix_lowres[...])
tifffile.imsave('./mov_lowres_data.tiff', mov_lowres[...])
tifffile.imsave('./fix_highres_data.tiff', fix_highres[...])
tifffile.imsave('./mov_highres_data.tiff', mov_highres[...])

## Run the pipeline
---

Pre-packaged pipelines for specific projects or papers are implemented in the `bigstream.application_pipelines` module. These pipelines chain together multiple alignment steps from more fundamental bigstream alignment functions. To learn about those more fundamental alignment functions, you should later work through the `bigstream_intro_tutorial.ipynb`.

In [ ]:
# First, familiarize yourself with the easifish_registration_pipeline API
from bigstream.application_pipelines import easifish_registration_pipeline
print("easifish_alignment_pipeline\n", easifish_registration_pipeline.__doc__)

In [ ]:
from bigstream.application_pipelines import easifish_registration_pipeline

# Arguments for your distributed system
cluster_kwargs = {}

# run the pipeline
affine, deform, aligned = easifish_registration_pipeline(
    fix_lowres, fix_highres, mov_lowres, mov_highres,
    fix_lowres_spacing, fix_highres_spacing,
    mov_lowres_spacing, mov_highres_spacing,
    blocksize=[128,]*3,
    write_directory='./',
    cluster_kwargs=cluster_kwargs,
)

# the affine and deform are already saved to disk, but we also want to view the aligned
# result to make sure it worked.
# reformat the aligned data to open in fiji (or similar) - again this works for tutorial data
# but you would do this differently for actually larger-than-memory data
tifffile.imsave('./aligned.tiff', aligned[...])